In [1]:
import os
import shutil
import glob
import numpy as np

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3))

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
batch_size = 32

# this is the augmentation configuration we will use for training
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [5]:
train_generator = datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224*224
        batch_size=batch_size,
        class_mode='categorical')
valid_generator = datagen.flow_from_directory(
        'data/valid',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 224*224
        batch_size=batch_size,
        class_mode='categorical')

Found 610 images belonging to 4 classes.
Found 107 images belonging to 4 classes.


In [6]:
i = 0
train_features = np.zeros(shape=(610, 7, 7, 512))
train_labels = np.zeros(shape=(610, 4))
for inputs_batch, labels_batch in train_generator:
    train_features_batch = conv_base.predict(inputs_batch)
    print (i*batch_size, np.minimum((i + 1) * batch_size,610))
    train_features[i * batch_size : np.minimum((i + 1) * batch_size,610)] = train_features_batch
    train_labels[i * batch_size : np.minimum((i + 1) * batch_size,610)] = labels_batch
    i += 1
    if i * batch_size >= 610:
        break

0 32
32 64
64 96
96 128
128 160
160 192
192 224
224 256
256 288
288 320
320 352
352 384
384 416
416 448
448 480
480 512
512 544
544 576
576 608
608 610


In [7]:
i = 0
valid_features = np.zeros(shape=(107, 7, 7, 512))
valid_labels = np.zeros(shape=(107, 4))
for inputs_batch, labels_batch in valid_generator:
    valid_features_batch = conv_base.predict(inputs_batch)
    print (i*batch_size, np.minimum((i + 1) * batch_size,107))
    valid_features[i * batch_size : np.minimum((i + 1) * batch_size,107)] = valid_features_batch
    valid_labels[i * batch_size : np.minimum((i + 1) * batch_size,107)] = labels_batch
    i += 1
    if i * batch_size >= 107:
        break

0 32
32 64
64 96
96 107


In [8]:
train_features=np.reshape(train_features,(610,7*7*512))
valid_features=np.reshape(valid_features,(107,7*7*512))

In [11]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7*7*512))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=32,
                    validation_data=(valid_features, valid_labels))

Train on 610 samples, validate on 107 samples
Epoch 1/30
610/610 [==============================] - 2s - loss: 1.1530 - acc: 0.5213 - val_loss: 0.9860 - val_acc: 0.6262